In [1]:
import sys

sys.path.append("..")

from src.auth.aurora import MySQL

In [2]:
mySQL = MySQL()

In [3]:
si_clients = [
    {'id': 4446, 'name': 'Allied Fire Protection Ltd'},
    {'id': 10028, 'name': 'Gentek Fire And Security Ltd'},
    {'id': 414, 'name': 'Global Fire & Security Systems'},
    {'id': 4056, 'name': 'SRS Alert Fire Systems Ltd'},
    {'id': 310, 'name': 'Tecserv UK Ltd'},
    {'id': 13, 'name': 'Trinity Fire & Security Systems'},
    {'id': 414, 'name': 'Global Fire & Security Systems'},
    {'id': 7650, 'name': 'Churches Fire Security Ltd'},
    {'id': 118, 'name': 'MasterFire'},
    {'id': 17, 'name': 'Sunfish Services'},
    {'id': 33, 'name': 'ADT Fire & Security PLC'},
    {'id': 94, 'name': 'Firesec Compliance Limited'},
    {'id': 10028, 'name': 'Gentek Fire And Security Ltd'},
    {'id': 7650, 'name': 'Churches Fire Security Ltd'},
    {'id': 87, 'name': 'Marlowe Fire & Security'},
 ]

In [4]:
client_domains = {}
for client in si_clients:
    id = client['id']
    query = """
    WITH RECURSIVE category_path AS (
        SELECT id, name, parent_id
        FROM api_clients
        WHERE id = %(client_id)s
        AND deleted_at IS NULL

        UNION ALL

        SELECT c.id, c.name, c.parent_id
        FROM api_clients c
        INNER JOIN category_path cp ON cp.id = c.parent_id
        AND deleted_at IS NULL
    )
    SELECT DISTINCT users.id, users.email
    FROM category_path AS clients
    LEFT JOIN api_collaborators AS collabs
    ON clients.id = collabs.client_id
    LEFT JOIN api_users as users
    ON collabs.user_id = users.id;
    """
    users_df = mySQL.query_to_pandas(
        query=query,
        params={
            "client_id": client['id']
        },
    )
    users_df["domain"] = users_df["email"].str.extract(r'((?<=@)[^.]+(?:\.[^.]+)+)')
    users_df.groupby("domain").size().sort_values(ascending=False).to_csv(
        f"""data/2025-06-02 Users Outside Domain/{client["name"]}.csv""",
        header=False
    )